<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=270744376" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv
/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kagg

In [2]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-v4/user_course_v4.csv'
df_user_video_final_path = '/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv'

In [3]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg, regexp_replace
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/25 10:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [5]:
df_user_video=spark.read.json(BASE_DIR + "/user-video.json")
# df_user_video.show()

In [6]:
# df_user_video.printSchema()

In [7]:
from pyspark.sql.functions import col, explode, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Giải nén `seq` để mỗi dòng có một `video_id`
df_flatten = df_user_video.withColumn("seq", explode(col("seq")))

# Giải nén tiếp `segment` để mỗi dòng là một đoạn xem video
df_flatten = df_flatten.withColumn("segment", explode(col("seq.segment")))

# Chọn các cột cần thiết
df_flatten = df_flatten.select(
    col("user_id"),
    col("seq.video_id").alias("video_id"),
    col("segment.start_point"),
    col("segment.end_point"),
    col("segment.speed"),
    col("segment.local_start_time")
)
# df_flatten.show(truncate=True)

In [8]:
from pyspark.sql.functions import from_unixtime

df_flatten = df_flatten.withColumn("local_start_time", from_unixtime("local_start_time", "yyyy-MM-dd HH:mm:ss"))
# df_flatten.show(truncate=False)

In [9]:
from pyspark.sql.functions import regexp_replace, col
df_cleaned = df_flatten.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

# df_cleaned.show(5, truncate=False)


In [10]:
df_cleaned.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- video_id: long (nullable = true)
 |-- start_point: double (nullable = true)
 |-- end_point: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- local_start_time: string (nullable = true)



In [11]:
df_cleaned = df_cleaned.filter(col("start_point") <= col("end_point"))


In [12]:
df_course = spark.read.json(BASE_DIR + '/course.json')
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)
df_video_course_map_clean = df_video_course_map.withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

# df_video_course_map_clean.show()

In [13]:
df_video = spark.read.json(BASE_DIR + "/video.json")
from pyspark.sql.functions import array_max
df_video = (
    df_video
    .select(
        "ccid", 
        array_max("end").alias("video_duration")
    )
)
# df_video.show()

In [14]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
# Parse mapping file
df_video_id_ccid_v2 = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
video_info = df_video_id_ccid_v2.join(df_video, on=['ccid'])
video_info_cleaned = video_info.withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)
video_info_cleaned.show(truncate=False)

+--------------------------------+--------+--------------+
|ccid                            |video_id|video_duration|
+--------------------------------+--------+--------------+
|035503AF2FB5812C9C33DC5901307461|125885  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|126437  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|126621  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|681656  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|793328  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|793761  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|794195  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|1260909 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2357873 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2358059 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2612910 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2748021 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2748207 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2971514 |1051.907     

In [15]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course_clean = df_user_course.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
)

# df_user_course_clean.show()

In [16]:
count = df_user_course_clean.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_course_clean.columns
])
# count.show(truncate=False)

In [17]:
df_uv_uc =  df_user_course_clean.join(
    df_cleaned,
    on='user_id',
    how='left'
)
df_uv_uc.show()

+-------+---------+-----------+----------+----------+-------------+-------------------+----------+--------+-----------+---------+-----+-------------------+
|user_id|course_id|certificate|start_date|  end_date|duration_days|        enroll_time|remain_day|video_id|start_point|end_point|speed|   local_start_time|
+-------+---------+-----------+----------+----------+-------------+-------------------+----------+--------+-----------+---------+-----+-------------------+
|  10000|  2033958|        1.0|2020-09-04|2020-12-31|        118.0|2020-10-27 09:07:30|      65.0|    NULL|       NULL|     NULL| NULL|               NULL|
| 100090|   754135|        1.0|2020-02-17|2020-07-26|        160.0|2020-03-23 09:35:23|     125.0|    NULL|       NULL|     NULL| NULL|               NULL|
| 100090|   737592|        1.0|2020-02-17|2020-07-05|        139.0|2020-03-23 09:38:38|     104.0|    NULL|       NULL|     NULL| NULL|               NULL|
| 100090|   681708|        1.0|2020-02-24|2020-07-31|        158

In [18]:
df_uv_uc.count()

37695451

In [19]:
count = df_uv_uc.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_uv_uc.columns
])
# count.show(truncate=False)

In [20]:
# import shutil

# input_dir = "/kaggle/working/df_final_output"
# output_zip = "/kaggle/working/df_final_output"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_final_output.zip")


In [21]:
# !rm -rf /kaggle/working/*
